<a href="https://colab.research.google.com/github/paratesai316/Benign-vs-Malignant-Classification/blob/main/Vision_Reader_Early_Concept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
Vision Reader App Early Concept

This file shows the early concept of the Vision Reader App.
It used 2 APIs: OCR API and Bytez API.
OCR API converts the image to text and Bytez API converts this text to audio.
This was the initial concept of Vision Reader.
This was before the stage of having Real-time camera access and scanning of the text.
```



In [2]:
pip install bytez

In [3]:
pip install --upgrade transformers accelerate;

In [4]:
pip install ocrspace

In [5]:
import requests
import ocrspace
from google.colab import files
import json
from bytez import Bytez
from IPython.display import Audio
from transformers import VitsModel, AutoTokenizer
import torch

In [6]:
filename = files.upload()

Saving default.trial.png to default.trial.png


In [7]:
ocrapi = '###' # Enter your OCR API key here
bytezapi = "###" # Enter your Bytez API key here
sdk = Bytez(bytezapi)
model_name = "Baghdad99/english_voice_tts"
model = VitsModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

In [8]:
def ocr_space_file(filename, overlay=False, api_key=ocrapi, language='eng'):
    """ OCR.space API request with local file.
        Python3.5 - not tested on 2.7
    :param filename: Your file path & name.
    :param overlay: Is OCR.space overlay required in your response.
                    Defaults to False.
    :param api_key: OCR.space API key.
                    Defaults to 'helloworld'.
    :param language: Language code to be used in OCR.
                    List of available language codes can be found on https://ocr.space/OCRAPI
                    Defaults to 'en'.
    :return: Result in JSON format.
    """

    payload = {'isOverlayRequired': overlay,
               'apikey': api_key,
               'language': language,
               }
    with open(filename, 'rb') as f:
        r = requests.post('https://api.ocr.space/parse/image',
                          files={filename: f},
                          data=payload,
                          )
    return r.content.decode()
    #credits to https://github.com/Zaargh/ocr.space_code_example

In [9]:
def extract_and_format_text(ocr_output):
    """
    Extracts text from OCR output by removin contents of JSON and maintaining only required text
    """
    try:
        data = json.loads(ocr_output)
        if data and data.get("ParsedResults"):
            parsed_text = data["ParsedResults"][0].get("ParsedText", "")
            formatted_text = parsed_text.replace('\r\n', ' ').strip()
            return formatted_text
        else:
            return "Could not text in image."
    except json.JSONDecodeError:
        return "System Error."

In [10]:
filename_str = list(filename.keys())[0]
text = extract_and_format_text(ocr_space_file(filename_str, overlay=False, api_key= ocrapi, language='eng'))
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    output_attentions = False
    output_hidden_states = False
    output = model(**inputs, output_attentions=output_attentions, output_hidden_states=output_hidden_states).waveform

audio_output = Audio(output.cpu().numpy(), rate=model.config.sampling_rate)
display(audio_output)
print(text)

The Trunchbull lifted the water-jug and poured some water into her glass. And suddenly, with the water, out came the long slimy newt straight into the glass, plop! The Trunchbull let out a yell and leapt off her chair as though a firecracker had gone off underneath her.
